In [11]:
# -*- coding: utf-8 -*-
"""
02_als_modeling_and_recommendations.ipynb

Ce notebook est dédié à la modélisation ALS pour les recommandations de films.
Il entraîne deux modèles ALS (un global, un sur les films récents),
effectue un Grid Search pour l'optimisation des hyperparamètres,
et évalue leur performance.
"""

# Importations nécessaires
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, when, lit, collect_list
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


In [2]:

# --- Initialisation de la SparkSession ---
print("--- Démarrage de la SparkSession pour la modélisation ALS ---")
spark = SparkSession.builder \
    .appName("ALSModeling") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.maxResults", 10)

--- Démarrage de la SparkSession pour la modélisation ALS ---


In [3]:
# --- Chargement des DataFrames préparés ---
print("Chargement des DataFrames préparés (training_data, test_data, df_movies_cleaned)...")
output_dir = "data" # Assurez-vous que ce chemin correspond à celui du notebook précédent

try:
    training_data = spark.read.parquet(os.path.join(output_dir, "training_data.parquet"))
    test_data = spark.read.parquet(os.path.join(output_dir, "test_data.parquet"))
    df_movies_cleaned = spark.read.parquet(os.path.join(output_dir, "df_movies_cleaned.parquet"))
    print("DataFrames chargés avec succès.")
    
    # Vérification rapide des données chargées
    print("\nSchéma et 5 premières lignes de training_data:")
    training_data.printSchema()
    training_data.show(5)
    print("\nSchéma et 5 premières lignes de test_data:")
    test_data.printSchema()
    test_data.show(5)
    print("\nSchéma et 5 premières lignes de df_movies_cleaned:")
    df_movies_cleaned.printSchema()
    df_movies_cleaned.show(5)

except Exception as e:
    print(f"Erreur lors du chargement des fichiers Parquet : {e}")
    print("Vérifiez que les fichiers Parquet ont bien été sauvegardés dans le dossier 'data/' par le notebook 01.")
    print("Le problème HADOOP_HOME doit être résolu pour cette étape.")
    spark.stop()
    exit() # Quitte le script si les données ne peuvent pas être chargées



Chargement des DataFrames préparés (training_data, test_data, df_movies_cleaned)...
DataFrames chargés avec succès.

Schéma et 5 premières lignes de training_data:
root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    260|   5.0| 943228696|
|     3|   7143|   4.0|1084485499|
|     6|   2949|   5.0|1100060102|
|     9|   1639|   3.5|1138474089|
|    10|   2231|   3.5|1190332092|
+------+-------+------+----------+
only showing top 5 rows


Schéma et 5 premières lignes de test_data:
root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|    16|  84374| 

In [4]:
# --- Configuration commune pour ALS et l'évaluation ---
user_col = "userId"
item_col = "movieId"
rating_col = "rating"
evaluator = RegressionEvaluator(metricName="rmse", labelCol=rating_col, predictionCol="prediction")

# --- Modèle ALS 1: Sur l'intégralité des données (training_data) ---
print("\n--- Modèle ALS (Global - sur toutes les données d'entraînement) ---")

als_global = ALS(
    userCol=user_col,
    itemCol=item_col,
    ratingCol=rating_col,
    coldStartStrategy="drop", # Gère les utilisateurs/films non vus dans le train set lors de la prédiction
    seed=42
)

# Définition de la grille de paramètres pour le Grid Search du modèle global
# Ces valeurs sont des points de départ. Ajustez selon vos ressources et le temps disponible.
param_grid_global = ParamGridBuilder() \
    .addGrid(als_global.rank, [10, 20]) \
    .addGrid(als_global.regParam, [0.01, 0.1]) \
    .addGrid(als_global.maxIter, [5, 10]) \
    .build()

cross_validator_global = CrossValidator(
    estimator=als_global,
    estimatorParamMaps=param_grid_global,
    evaluator=evaluator,
    numFolds=3, # Réduit le nombre de plis pour accélérer, mais 5 est plus robuste
    seed=42,
    parallelism=4 # Ajustez selon le nombre de cœurs de votre machine/cluster
)

print("Lancement du Grid Search pour le modèle ALS global (peut prendre du temps)...")
cv_model_global = cross_validator_global.fit(training_data)
best_model_global = cv_model_global.bestModel

print(f"\nMeilleurs hyperparamètres pour ALS Global - Rank: {best_model_global._java_obj.parent().getRank()}, RegParam: {best_model_global._java_obj.parent().getRegParam()}, MaxIter: {best_model_global._java_obj.parent().getMaxIter()}")




--- Modèle ALS (Global - sur toutes les données d'entraînement) ---
Lancement du Grid Search pour le modèle ALS global (peut prendre du temps)...

Meilleurs hyperparamètres pour ALS Global - Rank: 20, RegParam: 0.1, MaxIter: 10


In [5]:

# Évaluation du meilleur modèle ALS global sur le jeu de test
predictions_global = best_model_global.transform(test_data)
rmse_global = evaluator.evaluate(predictions_global.na.drop()) # Suppression des NaN pour le calcul RMSE

print(f"RMSE du meilleur modèle ALS Global sur le jeu de test : {rmse_global:.4f}")

# --- Sauvegarde du modèle ALS Global (utilisé plus tard pour KNN) ---
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)
best_model_global_path = os.path.join(models_dir, "best_als_global_model")
best_model_global.save(best_model_global_path)
print(f"\nMeilleur modèle ALS Global sauvegardé à : {best_model_global_path}")


RMSE du meilleur modèle ALS Global sur le jeu de test : 0.8327

Meilleur modèle ALS Global sauvegardé à : models\best_als_global_model


In [6]:
# --- Modèle ALS 2: Sur les films depuis 2018 ---
print("\n--- Modèle ALS (Films depuis 2018) ---")

# 1. Filtrer les notes pour n'inclure que les films sortis depuis 2018
# Joindre training_data avec df_movies_cleaned pour obtenir l'année de sortie
training_data_recent_movies = training_data.join(
    df_movies_cleaned.filter(col("release_year") >= 2018).select("movieId", "release_year"),
    on="movieId",
    how="inner"
)
# Assurez-vous que le DataFrame filtré n'est pas vide
if training_data_recent_movies.count() == 0:
    print("ATTENTION: Aucun film sorti depuis 2018 trouvé dans le jeu d'entraînement. Impossible d'entraîner le modèle récent.")
else:
    als_recent = ALS(
        userCol=user_col,
        itemCol=item_col,
        ratingCol=rating_col,
        coldStartStrategy="drop",
        seed=42
    )

    # Définition de la grille de paramètres pour le Grid Search du modèle récent
    param_grid_recent = ParamGridBuilder() \
        .addGrid(als_recent.rank, [8, 15]) \
        .addGrid(als_recent.regParam, [0.05, 0.15]) \
        .addGrid(als_recent.maxIter, [5, 10]) \
        .build()

    cross_validator_recent = CrossValidator(
        estimator=als_recent,
        estimatorParamMaps=param_grid_recent,
        evaluator=evaluator,
        numFolds=3,
        seed=42,
        parallelism=4
    )

    print("Lancement du Grid Search pour le modèle ALS sur les films récents (peut prendre du temps)...")
    cv_model_recent = cross_validator_recent.fit(training_data_recent_movies)
    best_model_recent = cv_model_recent.bestModel

    print(f"\nMeilleurs hyperparamètres pour ALS (Films depuis 2018) - Rank: {best_model_recent._java_obj.parent().getRank()}, RegParam: {best_model_recent._java_obj.parent().getRegParam()}, MaxIter: {best_model_recent._java_obj.parent().getMaxIter()}")

    # Évaluation du meilleur modèle ALS récent sur un sous-ensemble du jeu de test (films récents)
    test_data_recent_movies = test_data.join(
        df_movies_cleaned.filter(col("release_year") >= 2018).select("movieId", "release_year"),
        on="movieId",
        how="inner"
    )

    if test_data_recent_movies.count() > 0:
        predictions_recent = best_model_recent.transform(test_data_recent_movies)
        rmse_recent = evaluator.evaluate(predictions_recent.na.drop())
        print(f"RMSE du meilleur modèle ALS (Films depuis 2018) sur le jeu de test récent : {rmse_recent:.4f}")
    else:
        print("Pas de films récents dans le jeu de test pour évaluer le modèle récent.")



--- Modèle ALS (Films depuis 2018) ---
Lancement du Grid Search pour le modèle ALS sur les films récents (peut prendre du temps)...

Meilleurs hyperparamètres pour ALS (Films depuis 2018) - Rank: 15, RegParam: 0.15, MaxIter: 10
RMSE du meilleur modèle ALS (Films depuis 2018) sur le jeu de test récent : 2.6274


In [12]:
# --- Génération et affichage des recommandations ALS ---
# Nous allons utiliser le best_model_global pour générer les recommandations pour les utilisateurs fictifs,
# car c'est le modèle le plus complet.

print("\n--- Génération des recommandations ALS pour des utilisateurs fictifs ---")
users_to_recommend = [50, 150, 250, 350, 450] # Les mêmes utilisateurs que précédemment

# Créer un DataFrame des utilisateurs pour lesquels générer des recommandations
users_df = spark.createDataFrame([(user_id,) for user_id in users_to_recommend], ["userId"])

# Générer les 10 meilleures recommandations pour ces utilisateurs
# S'assure que les utilisateurs existent bien dans le userFactors du modèle
users_in_model_factors = best_model_global.userFactors.withColumnRenamed("id", "userId").select("userId").distinct()
users_to_recommend_filtered = users_df.join(users_in_model_factors, on="userId", how="inner")

if users_to_recommend_filtered.count() == 0:
    print("Aucun des utilisateurs fictifs n'est présent dans le modèle ALS global. Impossible de générer des recommandations.")
else:
    # Obtenir les recommandations ALS
    als_recommendations = best_model_global.recommendForUserSubset(users_to_recommend_filtered, 10)

    # Filtrer les films déjà vus par les utilisateurs dans le jeu d'entraînement pour des recommandations plus réalistes
    # (Bien que ALS.recommendForUserSubset tente de faire cela, une vérification explicite peut être utile)
    
    # Pour chaque utilisateur, récupérer les films qu'il a déjà vus dans training_data
    seen_movies_by_user = training_data.withColumnRenamed("id", "userId").groupBy("userId").agg(collect_list("movieId").alias("seen_movies"))

    # Convertir les recommandations en un format plus facile à manipuler
    # Assurez-vous que la colonne 'recommendations' contient des structs avec 'movieId'
    # Renommer la colonne 'id' en 'userId' dans userFactors si nécessaire (vérification faite dans le notebook 03)
    
    # Processus pour afficher les recommandations individuelles
    for row in als_recommendations.collect():
        user_id = row.userId
        recs = row.recommendations

        # Extraire les movieIds des recommandations
        recommended_movie_ids = [r.movieId for r in recs]
        
        # Récupérer les titres et genres des films recommandés
        recommended_movies_details = df_movies_cleaned.filter(col("movieId").isin(recommended_movie_ids)) \
            .select("movieId", "title", "genres").toPandas()
        
        # Ajouter la prédiction de rating pour chaque film recommandé
        # Créer un dictionnaire pour mapper movieId à prediction
        prediction_map = {r.movieId: r.rating for r in recs}
        recommended_movies_details['prediction'] = recommended_movies_details['movieId'].map(prediction_map)

        # Trier par la prédiction ALS et afficher
        recommended_movies_details = recommended_movies_details.sort_values(by='prediction', ascending=False)
        
        print(f"\n================== RECOMMANDATIONS ALS POUR L'UTILISATEUR {user_id} ==================")
        print(recommended_movies_details[['title', 'genres', 'prediction']].to_string(index=False))



--- Génération des recommandations ALS pour des utilisateurs fictifs ---

================== RECOMMANDATIONS ALS POUR L'UTILISATEUR 50 ==================
                        title                  genres  prediction
                    The Thorn                  Comedy    6.290634
                      Acı Aşk                   Drama    6.112364
                  Head Trauma Horror|Mystery|Thriller    5.621753
              Catch That Girl         Action|Children    5.579004
                            2                   Drama    5.528359
 Life Even Looks Like a Party             Documentary    5.379093
             Shivering Trunks      (no genres listed)    5.337296
                         Loot            Comedy|Crime    5.336211
Christmas on Salvation Street      (no genres listed)    5.328463
        Story of Science, The             Documentary    5.317255

================== RECOMMANDATIONS ALS POUR L'UTILISATEUR 450 ==================
                                     

In [ ]:
# --- Arrêt de la SparkSession ---
print("\n--- Arrêt de la SparkSession ---")
spark.stop()

print("\n--- Script 02_als_modeling_and_recommendations.ipynb Terminé ---")